# 训练数据生成器

本notebook用于生成LSTM波形预测模型的训练数据，并将数据保存为DGR格式到`src/ml/data`目录中。

## 功能特性
- **批量数据生成**: 创建多个录制会话用于训练
- **DGR格式保存**: 使用DGR (DG-Lab Recording) 格式保存数据
- **数据验证**: 验证生成数据的完整性和正确性
- **灵活配置**: 支持自定义数据生成参数
- **文件管理**: 每个会话保存为独立的DGR文件


In [6]:
# 导入必要的库
import sys
import os

sys.path.append(os.path.join(os.getcwd(), '..'))

from pathlib import Path
from typing import List, Optional
from datetime import datetime

# 导入数据模型和DGR文件管理器
from models import Channel
from core.recording.recording_models import RecordingSession, RecordingMetadata, RecordingSnapshot, ChannelSnapshot
from core.recording.dgr_file_manager import DGRFileManager

print("✅ 所有库导入成功！")
print(f"当前工作目录: {os.getcwd()}")

# 确保数据目录存在
data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)
print(f"数据目录: {data_dir.absolute()}")

# 创建DGR文件管理器实例
dgr_manager = DGRFileManager()
print("✅ DGR文件管理器初始化完成！")


✅ 所有库导入成功！
当前工作目录: e:\projects\DG-LAB-VRCOSC\src\ml
数据目录: e:\projects\DG-LAB-VRCOSC\src\ml\data
✅ DGR文件管理器初始化完成！


In [7]:
# 定义创建示例会话的辅助函数
def create_sample_session(session_id: Optional[str] = None, duration_ms: int = 10000) -> RecordingSession:
    """
    创建示例录制会话用于测试
    
    Args:
        session_id: 会话ID，如果为None则自动生成
        duration_ms: 录制持续时间（毫秒）
    
    Returns:
        RecordingSession: 生成的录制会话
    """
    if session_id is None:
        session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # 创建示例元数据
    metadata = RecordingMetadata(
        session_id=session_id,
        start_time=datetime.now()
    )
    
    # 计算快照数量（假设每100ms一个快照）
    num_snapshots = duration_ms // 100
    
    # 创建示例快照数据
    snapshots: List[RecordingSnapshot] = []
    for i in range(num_snapshots):
        # 创建A通道脉冲操作 - 使用更真实的波形模式
        # 确保频率在[10, 240]范围内，强度在[0, 100]范围内
        freq_a = (
            max(10, min(240, 50 + (i % 50) * 2)),  # 基础频率50Hz，逐渐增加到150Hz
            max(10, min(240, 60 + (i % 40) * 1)),  # 第二频率
            max(10, min(240, 70 + (i % 30) * 1)),  # 第三频率
            max(10, min(240, 80 + (i % 20) * 1))   # 第四频率
        )
        strength_a = (
            max(0, min(100, 20 + (i % 30) * 1)),  # 基础强度20%，逐渐增加到50%
            max(0, min(100, 30 + (i % 25) * 1)),  # 第二强度
            max(0, min(100, 40 + (i % 20) * 1)),  # 第三强度
            max(0, min(100, 50 + (i % 15) * 1))   # 第四强度
        )
        pulse_a = (freq_a, strength_a)
        
        # 创建B通道脉冲操作 - 使用不同的波形模式
        freq_b = (
            max(10, min(240, 40 + (i % 60) * 1)),  # 基础频率40Hz，逐渐增加到100Hz
            max(10, min(240, 50 + (i % 50) * 1)),  # 第二频率
            max(10, min(240, 60 + (i % 40) * 1)),  # 第三频率
            max(10, min(240, 70 + (i % 30) * 1))   # 第四频率
        )
        strength_b = (
            max(0, min(100, 15 + (i % 35) * 1)),  # 基础强度15%，逐渐增加到50%
            max(0, min(100, 25 + (i % 30) * 1)),  # 第二强度
            max(0, min(100, 35 + (i % 25) * 1)),  # 第三强度
            max(0, min(100, 45 + (i % 20) * 1))   # 第四强度
        )
        pulse_b = (freq_b, strength_b)
        
        # 创建通道快照 - 确保current_strength在[0, 200]范围内
        channel_a = ChannelSnapshot(pulse_a, max(0, min(200, 50 + (i % 100))))
        channel_b = ChannelSnapshot(pulse_b, max(0, min(200, 60 + (i % 80))))
        
        # 创建录制快照
        snapshot = RecordingSnapshot({
            Channel.A: channel_a,
            Channel.B: channel_b
        })
        snapshots.append(snapshot)
    
    return RecordingSession(metadata, snapshots)

print("✅ 数据生成函数定义完成！")


✅ 数据生成函数定义完成！


In [8]:
# 数据保存和加载函数
async def save_training_sessions(sessions: List[RecordingSession], filename: str) -> None:
    """
    保存训练会话数据到DGR文件
    
    Args:
        sessions: 录制会话列表
        filename: 保存的文件名（不包含.dgr扩展名）
    """
    # 使用DGR文件管理器保存每个会话
    for i, session in enumerate(sessions):
        session_file_path = data_dir / f"{filename}_{i+1:03d}.dgr"
        await dgr_manager.save_recording(session, str(session_file_path))
    
    print(f"✅ 训练数据已保存到DGR文件: {data_dir}")
    print(f"   会话数量: {len(sessions)}")
    print(f"   总快照数量: {sum(len(session.snapshots) for session in sessions)}")
    print(f"   文件格式: DGR (DG-Lab Recording)")

async def load_training_sessions(filename_prefix: str) -> List[RecordingSession]:
    """
    从DGR文件加载训练会话数据
    
    Args:
        filename_prefix: 文件名前缀（不包含.dgr扩展名）
        
    Returns:
        List[RecordingSession]: 加载的录制会话列表
    """
    sessions: List[RecordingSession] = []
    
    # 查找所有匹配的DGR文件
    pattern = f"{filename_prefix}_*.dgr"
    dgr_files = list(data_dir.glob(pattern))
    dgr_files.sort()  # 确保按顺序加载
    
    if not dgr_files:
        print(f"❌ 未找到匹配的DGR文件: {pattern}")
        return sessions
    
    for dgr_file in dgr_files:
        try:
            session = await dgr_manager.load_recording(str(dgr_file))
            sessions.append(session)
        except Exception as e:
            print(f"⚠️ 加载文件失败 {dgr_file}: {e}")
    
    print(f"✅ 训练数据已从DGR文件加载")
    print(f"   会话数量: {len(sessions)}")
    print(f"   总快照数量: {sum(len(session.snapshots) for session in sessions)}")
    print(f"   文件格式: DGR (DG-Lab Recording)")
    
    return sessions

In [9]:
# 生成训练数据
print("🚀 开始生成训练数据...")

# 配置参数
NUM_TRAINING_SESSIONS = 10  # 训练会话数量
NUM_VALIDATION_SESSIONS = 3  # 验证会话数量
SESSION_DURATION_MS = 10000  # 每个会话持续时间（毫秒）

print(f"配置参数:")
print(f"  - 训练会话数量: {NUM_TRAINING_SESSIONS}")
print(f"  - 验证会话数量: {NUM_VALIDATION_SESSIONS}")
print(f"  - 会话持续时间: {SESSION_DURATION_MS}ms")

# 生成训练会话
print("\n📚 生成训练会话...")
training_sessions: List[RecordingSession] = []
for i in range(NUM_TRAINING_SESSIONS):
    session = create_sample_session(
        session_id=f"training_session_{i+1:03d}",
        duration_ms=SESSION_DURATION_MS
    )
    training_sessions.append(session)

print(f"✅ 生成了 {len(training_sessions)} 个训练会话")

# 生成验证会话
print("\n🔍 生成验证会话...")
validation_sessions: List[RecordingSession] = []
for i in range(NUM_VALIDATION_SESSIONS):
    session = create_sample_session(
        session_id=f"validation_session_{i+1:03d}",
        duration_ms=SESSION_DURATION_MS
    )
    validation_sessions.append(session)

print(f"✅ 生成了 {len(validation_sessions)} 个验证会话")

# 显示数据统计
total_snapshots = sum(len(session.snapshots) for session in training_sessions + validation_sessions)
print(f"\n📊 数据统计:")
print(f"  - 总会话数量: {len(training_sessions + validation_sessions)}")
print(f"  - 总快照数量: {total_snapshots}")
print(f"  - 平均每会话快照数: {total_snapshots / len(training_sessions + validation_sessions):.1f}")

# 显示第一个会话的详细信息
first_session = training_sessions[0]
print(f"\n🔍 第一个训练会话详情:")
print(f"  - 会话ID: {first_session.metadata.session_id}")
print(f"  - 快照数量: {len(first_session.snapshots)}")
print(f"  - 持续时间: {first_session.get_duration_ms()}ms")

# 显示第一个快照的详细信息
first_snapshot = first_session.snapshots[0]
print(f"\n📋 第一个快照详情:")
for channel in [Channel.A, Channel.B]:
    if channel in first_snapshot.channels:
        channel_snapshot = first_snapshot.channels[channel]
        pulse_op = channel_snapshot.pulse_operation
        freq_op, strength_op = pulse_op
        print(f"  - 通道{channel.name}: 频率={freq_op}, 强度={strength_op}, 当前强度={channel_snapshot.current_strength}")


🚀 开始生成训练数据...
配置参数:
  - 训练会话数量: 10
  - 验证会话数量: 3
  - 会话持续时间: 10000ms

📚 生成训练会话...
✅ 生成了 10 个训练会话

🔍 生成验证会话...
✅ 生成了 3 个验证会话

📊 数据统计:
  - 总会话数量: 13
  - 总快照数量: 1300
  - 平均每会话快照数: 100.0

🔍 第一个训练会话详情:
  - 会话ID: training_session_001
  - 快照数量: 100
  - 持续时间: 10000ms

📋 第一个快照详情:
  - 通道A: 频率=(50, 60, 70, 80), 强度=(20, 30, 40, 50), 当前强度=50
  - 通道B: 频率=(40, 50, 60, 70), 强度=(15, 25, 35, 45), 当前强度=60


In [10]:
# 保存训练数据
print("\n💾 保存训练数据到DGR文件...")

# 保存训练会话
training_filename = "training_sessions"
print("保存训练会话...")
await save_training_sessions(training_sessions, training_filename)

# 保存验证会话
validation_filename = "validation_sessions"
print("保存验证会话...")
await save_training_sessions(validation_sessions, validation_filename)

# 保存数据配置信息
import json
config_data = {
    "generation_time": datetime.now().isoformat(),
    "num_training_sessions": len(training_sessions),
    "num_validation_sessions": len(validation_sessions),
    "session_duration_ms": SESSION_DURATION_MS,
    "total_snapshots": total_snapshots,
    "files": {
        "training_prefix": training_filename,
        "validation_prefix": validation_filename
    },
    "format": "DGR (DG-Lab Recording)",
    "version": "1.0"
}

config_filename = "data_config.json"
config_path = data_dir / config_filename
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config_data, f, ensure_ascii=False, indent=2)

print(f"✅ 数据配置已保存到: {config_path}")

# 显示保存的文件信息
print(f"\n📁 保存的文件:")
print(f"  - 训练数据: {data_dir}/*{training_filename}_*.dgr")
print(f"  - 验证数据: {data_dir}/*{validation_filename}_*.dgr")
print(f"  - 配置文件: {config_path}")

# 验证数据完整性
print(f"\n🔍 验证数据完整性...")
loaded_training: List[RecordingSession] = await load_training_sessions(training_filename)
loaded_validation: List[RecordingSession] = await load_training_sessions(validation_filename)

print(f"✅ 数据验证完成!")
print(f"  - 训练数据加载成功: {len(loaded_training)} 个会话")
print(f"  - 验证数据加载成功: {len(loaded_validation)} 个会话")

print(f"\n🎉 训练数据生成和保存完成！")
print(f"   数据已保存到: {data_dir.absolute()}")
print(f"   文件格式: DGR (DG-Lab Recording)")
print(f"   可以开始训练LSTM模型了！")



💾 保存训练数据到DGR文件...
保存训练会话...
✅ 训练数据已保存到DGR文件: data
   会话数量: 10
   总快照数量: 1000
   文件格式: DGR (DG-Lab Recording)
保存验证会话...
✅ 训练数据已保存到DGR文件: data
   会话数量: 3
   总快照数量: 300
   文件格式: DGR (DG-Lab Recording)
✅ 数据配置已保存到: data\data_config.json

📁 保存的文件:
  - 训练数据: data/*training_sessions_*.dgr
  - 验证数据: data/*validation_sessions_*.dgr
  - 配置文件: data\data_config.json

🔍 验证数据完整性...
✅ 训练数据已从DGR文件加载
   会话数量: 10
   总快照数量: 1000
   文件格式: DGR (DG-Lab Recording)
✅ 训练数据已从DGR文件加载
   会话数量: 3
   总快照数量: 300
   文件格式: DGR (DG-Lab Recording)
✅ 数据验证完成!
  - 训练数据加载成功: 10 个会话
  - 验证数据加载成功: 3 个会话

🎉 训练数据生成和保存完成！
   数据已保存到: e:\projects\DG-LAB-VRCOSC\src\ml\data
   文件格式: DGR (DG-Lab Recording)
   可以开始训练LSTM模型了！
